<a href="https://colab.research.google.com/github/AliGreo/Text-Based-Projects/blob/main/summarization_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers peft evaluate loralib datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer, GenerationConfig
import torch
import evaluate
from peft import LoraConfig
import pandas as pd
from datasets import load_dataset

In [48]:
dataset = load_dataset("knkarthick/dialogsum")

In [49]:
pd.DataFrame(dataset["train"])

,id,dialogue,summary,topic
0,train_0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,find keys
3,train_3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,have a girlfriend
4,train_4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,dance
...,...,...,...,...
12455,train_12455,#Person1#: Excuse me. You are Mr. Green from M...,Tan Ling picks Mr. Green up who is easily reco...,pick up someone
12456,train_12456,#Person1#: Mister Ewing said we should show up...,#Person1# and #Person2# plan to take the under...,conference center
12457,train_12457,#Person1#: How can I help you today?\n#Person2...,#Person2# rents a small car for 5 days with th...,rent a car
12458,train_12458,#Person1#: You look a bit unhappy today. What'...,#Person2#'s mom lost her job. #Person2# hopes ...,job losing


In [50]:
model_name= 'google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name,
                                              torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## using The base Model to generate summaries.

In [51]:
prompt = f"""

summarise the following dialogue:{dataset["test"][0]["dialogue"]}\n

summary: \n\n

"""

ids = tokenizer(prompt, return_tensors="pt").input_ids


output = model.generate(ids, max_new_tokens=100)
print(output, "\n\n")
generated_summary = tokenizer.decode(output[0], skip_special_tokens=True)

print(f"The original summary:\n {dataset['test'][0]['summary']}", "\n\n")
print(f"The generated summary:\n {generated_summary}")

tensor([[    0,    37, 22986,    19,    12,    36,  8308,    12,    66,  1652,
            57,    48,  3742,     5,     1]]) 


The original summary:
 Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore. 


The generated summary:
 The memo is to be distributed to all employees by this afternoon.


## Fine Tune The Model for more accurate results.

In [52]:
import numpy as np

def tokenize_dataset(example):
    start_prompt = "summarize the following conversation:"
    end_prompt = "summary:\n"
    full_prompt = [start_prompt + ex + end_prompt for ex in example["dialogue"]]

    example["input_ids"] = tokenizer(full_prompt, max_length=512,padding="max_length", truncation=True, return_tensors="pt").input_ids
    example["labels"] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    example['labels'] = np.where(example['labels'] != -100, example['labels'], tokenizer.pad_token_id)

    return example

tokenized_dataset = dataset.map(tokenize_dataset, batched=True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [53]:
tokenized_dataset = tokenized_dataset.remove_columns(['id', 'topic', 'dialogue', 'summary',])

In [54]:
del dataset, ids, output, generated_summary

In [55]:
print(tokenized_dataset['train'][3]['labels'])

[1713, 345, 13515, 536, 4663, 31, 7, 12603, 250, 1713, 345, 13515, 357, 4663, 737, 31, 17, 817, 1713, 345, 13515, 536, 4663, 24, 1713, 345, 13515, 357, 4663, 141, 3, 9, 17442, 11, 133, 20111, 160, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [56]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [57]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Matrix Rank
    lora_alpha=64,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

peft_model = get_peft_model(model,
                            lora_config)

print(peft_model.print_trainable_parameters())

trainable params: 3,538,944 || all params: 251,116,800 || trainable%: 1.4093
None


In [25]:
from huggingface_hub import notebook_login
notebook_login()

In [61]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    output_dir="flan-t5-base-finetuned-dialouge-summarization",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    fp16=True,
    num_train_epochs=1,
    logging_steps=500,
    push_to_hub= True
)

trainer = Seq2SeqTrainer(
    model=peft_model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    processing_class=tokenizer
)

In [62]:
trainer.train()

Step,Training Loss
500,0.000000
1000,0.000000
1500,0.000000
2000,0.000000
2500,0.000000
3000,0.000000
3500,0.000000
4000,0.000000
4500,0.000000
5000,0.000000


TrainOutput(global_step=6230, training_loss=0.0, metrics={'train_runtime': 1881.055, 'train_samples_per_second': 6.624, 'train_steps_per_second': 3.312, 'total_flos': 8667537195663360.0, 'train_loss': 0.0, 'epoch': 1.0})

In [63]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/AlyGreo/flan-t5-base-finetuned-dialouge-summarization/commit/5092bf3f077e6df87a3e844e5569664a1eb62639', commit_message='End of training', commit_description='', oid='5092bf3f077e6df87a3e844e5569664a1eb62639', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AlyGreo/flan-t5-base-finetuned-dialouge-summarization', endpoint='https://huggingface.co', repo_type='model', repo_id='AlyGreo/flan-t5-base-finetuned-dialouge-summarization'), pr_revision=None, pr_num=None)

In [64]:
model_id = "/content/flan-t5-base-finetuned-dialouge-summarization"
mytokenizer = AutoTokenizer.from_pretrained(model_id)

In [66]:
myModel = AutoModelForSeq2SeqLM.from_pretrained(model_id, torch_dtype=torch.bfloat16)

In [78]:
output = myModel.generate(input_ids=torch.tensor(tokenized_dataset['test'][0]['input_ids']).view(1, -1),
                          max_new_tokens=50)

print(mytokenizer.decode(output[0], skip_special_tokens=True))

Employees will be required to use instant messaging during working hours.


In [79]:
output = myModel.generate(input_ids=torch.tensor(tokenized_dataset['test'][100]['input_ids']).view(1, -1),
                          max_new_tokens=50)

print(mytokenizer.decode(output[0], skip_special_tokens=True))

The two men are trying to figure out how to react to a cut.
